In [42]:
import numpy as np
import cvxopt

In [43]:
Tcmd = 100
Tmax = [50, 50, 200, 200]
Tmin = [-t for t in Tmax]
L = [Tmax[0] / Tmax[2], Tmax[1] / Tmax[3], Tmax[0] / Tmax[1], Tmax[2] / Tmax[3]]
dT = 10

In [44]:

# Define the quadratic cost function Q
W = np.diag([1e-3, 1] + [1e-2 for _ in range(4)])

# Define the inequality constraints matrix A and vector b
A = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
])
b = np.array(Tmax)

# Define the quadratic and linear terms of the cost function
C = np.array([
    [1, 1, 1, 1],
    [1, -1, 1, -1],
    [1, 0, -L[0], 0],
    [0, 1, 0, -L[1]],
    [1, -L[2], 0, 0],
    [0, 0, 1, -L[3]]
])

ref = np.array([Tcmd, dT, 0, 0, 0, 0])

Q = C.T @ W @ C
c = -2 * ref @ W @ C


# Number of variables
n = Q.shape[0]

In [45]:
# Convert numpy arrays to cvxopt matrices
Q_cvx = cvxopt.matrix(Q)
c_cvx = cvxopt.matrix(c)
A_cvx = cvxopt.matrix(A.astype(float))
b_cvx = cvxopt.matrix(b.astype(float))

# Solve the quadratic program
solution = cvxopt.solvers.qp(P=Q_cvx, q=c_cvx, G=A_cvx, h=b_cvx)

# Extract the optimal values
x_opt = np.array(solution['x'])

print("Optimal solution:", x_opt.flatten())

     pcost       dcost       gap    pres   dres
 0: -1.6634e+02 -1.0888e+02  4e+01  3e-02  3e-01
 1: -1.8040e+02 -1.8269e+02  2e+00  2e-03  2e-02
 2: -1.8956e+02 -1.9113e+02  2e+00  2e-03  1e-02
 3: -2.1655e+02 -2.6232e+02  5e+01  5e-04  4e-03
 4: -2.1887e+02 -2.2120e+02  2e+00  1e-16  7e-16
 5: -2.1890e+02 -2.1893e+02  3e-02  6e-17  3e-16
 6: -2.1890e+02 -2.1891e+02  3e-04  2e-16  6e-16
 7: -2.1890e+02 -2.1890e+02  3e-06  5e-17  8e-16
Optimal solution found.
Optimal solution: [24.13453998 15.86545928 85.81070082 74.18929096]


In [46]:
# Formulate the Lagrangian matrices
n = Q.shape[0]
m = A.shape[0]

# Construct the augmented matrix
LHS = np.block([
    [Q, A.T],
    [A, np.zeros((m, m))]
])

# Construct the augmented vector
RHS = np.concatenate([-c, b])

# Check the rank of the augmented matrix
if np.linalg.matrix_rank(LHS) < LHS.shape[0]:
    print("Augmented matrix is rank-deficient. Consider checking constraints.")

# Solve the linear system using numpy
try:
    sol = np.linalg.solve(LHS, RHS)
    x_opt = sol[:n]  # optimal solution for x
    lambda_opt = sol[n:]  # optimal solution for lambda
    print("Optimal solution x:", x_opt)
    print("Optimal solution lambda:", lambda_opt)
except np.linalg.LinAlgError:
    print("Singular matrix error: Unable to solve the system.")

Optimal solution x: [ 50.  50. 200. 200.]
Optimal solution lambda: [ 19.7 -20.3  19.7 -20.3]


In [47]:
Q_inv = np.linalg.inv(Q)

def compute_x(lambda_):
    return -Q_inv @ (c + A.T @ lambda_)

# Second part: solve the inequality constraints
# A x <= b  => A (-Q_inv (c + A^T lambda)) <= b
# A Q_inv A^T lambda <= b + A Q_inv c
H = A @ Q_inv @ A.T
g = b + A @ Q_inv @ c

# Solve H lambda = g
lambda_ = np.linalg.solve(H, g)

# Compute the optimal x
x = compute_x(lambda_)

In [48]:
# Check primal feasibility: A x <= b
primal_feasibility = A @ x - b <= 1e-5  # Using a small tolerance for numerical precision

# Check complementary slackness: lambda_i (A x - b)_i = 0
complementary_slackness = np.allclose(lambda_ * (A @ x - b), 0, atol=1e-5)

# Print the results
print("Optimal x:")
print(x, sum(x), x @ np.array([1, -1, 1, -1]))
print("\nLagrange multipliers (lambda):")
print(lambda_)
print("\nPrimal feasibility (A x <= b):")
print(primal_feasibility)
print(A @ x, b)
print("\nComplementary slackness:")
print(complementary_slackness)

Optimal x:
[ -1.73091966 -18.26908034 -28.37858979 -51.62141021] -100.00000000001796 39.78098111521126

Lagrange multipliers (lambda):
[-19.7  20.3 -19.7  20.3]

Primal feasibility (A x <= b):
[ True  True  True  True]
[ -1.73091966 -18.26908034 -28.37858979 -51.62141021] [ 50  50 200 200]

Complementary slackness:
False
